In [2]:
import pandas as pd
import os

In [70]:
SAMPLES_DIR = './samples/New/'
TRIAL_DIRS = {
    'pcvr-0':'PCVR-ConfThresh_0/', 
    'pcvr-0.5':'PCVR-ConfThresh_0.5/',
    'battery-0':'Standalone-Battery-ConfThresh_0/', 
    'battery-0.5':'Standalone-Battery-ConfThresh_0.5/',
    'powered-0':'Standalone-PluggedIn-ConfThresh_0/', 
    'powered-0.5':'Standalone-PluggedIn-ConfThresh_0.5/'
}
EYE_FILES = {
    'control':'Left_Eye_Control.csv', 
    'minimal':'Left_Eye_3.csv', 
    'trial_1':'Left_Eye_2.csv', 
    'trial_2':'Left_Eye_1.csv'
}

def get_filepath(trial_key, eye_key):
    return os.path.join(SAMPLES_DIR, TRIAL_DIRS[trial_key], EYE_FILES[eye_key])

def read_trial(filepath:str, sec_from_end:float, sec_duration:float):
    # Read CSV
    df = pd.read_csv(filepath)
    
    # Get more sensible timestamps
    s = df['unix_ms'].iloc[0]
    f = df['frame'].iloc[0]
    df['trial_ms'] = df['unix_ms'] - s
    df['trial_sec'] = df['trial_ms'] / 1000
    df['trial_frame'] = df['frame'] - f
    
    # Filter by seconds from the end (sec) and and duration (sec). Then re-center the trial_ms and trial_sec
    end_timestamp = df['trial_sec'].iloc[-1] - sec_from_end
    df2 = df[df['trial_sec'].between(end_timestamp - sec_duration, end_timestamp)]
    s2 = df2['trial_ms'].iloc[0]
    s3 = df2['trial_sec'].iloc[0]
    f2 = df2['trial_frame'].iloc[0]
    df2['trial_ms2'] = df2['trial_ms'] - s2
    df2['trial_sec2'] = df2['trial_sec'] - s3
    df2['trial_frame2'] = df2['trial_frame'] - f2
    df3 = df2.drop(columns=['id','frame','unix_ms','trial_ms','trial_sec','trial_frame'])
    df4 = df3.rename(columns={'trial_ms2':'trial_ms', 'trial_sec2':'trial_sec', 'trial_frame2':'trial_frame'})
    
    # Sample Rate interpretation
    df4['sample_count'] = range(0, len(df4))
    
    # Diff Interpretation
    df4['rotation_x_diff'] = df4['rotation_x'].diff().fillna(0)
    df4['rotation_y_diff'] = df4['rotation_y'].diff().fillna(0)
    df4['rotation_z_diff'] = df4['rotation_z'].diff().fillna(0)
    df4['trial_frame_diff'] = df4['trial_frame'].diff().fillna(0)
    
    # Return results
    return df4

In [85]:
df = read_trial(get_filepath('battery-0','control'), 10, 30)
df_nonzero = df[df['rotation_x_diff'] != 0.0]
sample_rate = len(df_nonzero)/30
sample_rate

C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_18240\4127621514.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['trial_ms2'] = df2['trial_ms'] - s2
C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_18240\4127621514.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['trial_sec2'] = df2['trial_sec'] - s3
C:\Users\Ryan Kim\AppData\Local\Temp\ipykernel_18240\4127621514.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

71.76666666666667

In [87]:
frame_duration_sec = 30/len(df_nonzero)
frame_duration_ms = frame_duration_sec * 1000
print(frame_duration_sec, frame_duration_ms)

0.013934045517882025 13.934045517882025
